In [1]:
import io
import os
import csv
import gzip
import time
import multiprocessing
import resource
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import statsmodels.api as sm
import random
from collections import Counter
import matplotlib.colors as mcolors
from scipy.stats import poisson
import itertools
import collections
import scipy
from scipy.stats import chi2
from scipy.stats import friedmanchisquare
from scipy.stats import studentized_range
pd.options.mode.chained_assignment = None

In [2]:
def get_mem():
    current_memory_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    current_memory_usage_mb = current_memory_usage / 1024
    print(f"Current memory usage: {current_memory_usage_mb:.2f} MB")
def read_vcf(file):
    with io.TextIOWrapper(gzip.open(file,'r')) as f:
        lines =[l for l in f if not l.startswith('##')]
        dynamic_header_as_key = []
        for liness in f:
            if liness.startswith("#CHROM"):
                dynamic_header_as_key.append(liness)
        values = [str,int,str,str,str,int,str,str,str,str]
        columns2detype = dict(zip(dynamic_header_as_key,values))
        df = pd.read_csv(
            io.StringIO(''.join(lines)),
            dtype=columns2detype,
            sep='\t'
        ).rename(columns={'#CHROM':'CHROM'})
    df['CHROM'] = df['CHROM'].str.extract(r'(\d+)').astype(int)
    return df
def extract_info(df, info_cols = ['EAF', 'INFO_SCORE'], attribute = 'INFO', drop_attribute = True):
    for i in info_cols:
        df[i] = df[attribute].str.extract( i + '=([^;]+)' ).astype(float)
    if drop_attribute:
        df = df.drop(columns = [attribute])
    return df
def extract_format(df, sample, fmt = 'FORMAT'):
    fields = df[fmt].values[0].split(':')
    try:
        df[fields] = df[sample].str.split(':', expand=True)
        df[df.columns[-1]] = df[df.columns[-1]].astype(float)
        if len(fields) != len(df[sample].values[0].split(':')):
            raise ValueError("Mismatching fields in FORMAT and Imputed results.")
    except ValueError as e:
        print(f"Error: {e}")
    return df.drop(columns = [fmt, sample])
def drop_cols(df, drop_lst = ['ID', 'QUAL', 'FILTER']):
    return df.drop(columns = drop_lst)

def parse_vcf(file, sample, q = None, 
              info_cols = ['EAF', 'INFO_SCORE'], attribute = 'INFO', fmt = 'FORMAT', drop_attribute = True, drop_lst = ['ID', 'QUAL', 'FILTER']):
    df = read_vcf(file)
    df = extract_info(df, info_cols = info_cols, attribute = attribute, drop_attribute = drop_attribute)
    df = extract_format(df, sample, fmt = fmt)
    df = drop_cols(df, drop_lst = drop_lst)
    if q is None:
        return df
    else:
        q.put(df)
def concat_vcf(lst):
    df = lst[0]
    for i in range(1, len(lst)):
        df = pd.concat([df, lst[i]])
    return df.reset_index(drop = True)
def subtract_bed_by_chr(cov, region):
    i = 0
    tmp = 0
    for j in range(region.shape[0]):
        start, end = region.iloc[j,:]
        while start > cov.iloc[i,2]:
            i += 1
        if start < cov.iloc[i,1]:
            cov.iloc[i-1, 2] = start
            if end < cov.iloc[i,2]:
                cov.iloc[i,1] = end
            elif end == cov.iloc[i,2]:
                cov.iloc[i,3] = -9
                i += 1
            else:
                tmp = i
                while end > cov.iloc[tmp,2]:
                    tmp += 1
                if end < cov.iloc[tmp, 2]:
                    cov.iloc[tmp, 1] = end
                    cov.iloc[i:tmp, 3] = -9
                    i = tmp
                else: 
                    cov.iloc[i:tmp+1, 3] = -9
                    i = tmp
        elif start == cov.iloc[i,1]:
            if end < cov.iloc[i,2]:
                cov.iloc[i,1] = end
            elif end == cov.iloc[i,2]:
                cov.iloc[i, 3] = -9
            else:
                tmp = i
                while end > cov.iloc[tmp,2]:
                    tmp += 1
                if end < cov.iloc[tmp, 2]:
                    cov.iloc[tmp, 1] = end
                    cov.iloc[i:tmp+1, 3] = -9
                    i = tmp
                else: 
                    cov.iloc[i:tmp, 3] = -9
                    i = tmp
        else: 
            idx = cov.index.max() + 1
            cov.loc[idx] = {'chr': chr, 'start': cov.iloc[i,1], 'end': start, 'cov': cov.iloc[i,3]}
            if end < cov.iloc[i, 2]:
                cov.iloc[i, 1] = end
            elif end == cov.iloc[i, 2]:
                cov.iloc[i, 3] = -9
            else: 
                tmp = i
                while end > cov.iloc[tmp,2]:
                    tmp += 1
                if end < cov.iloc[tmp, 2]:
                    cov.iloc[tmp, 1] = end
                    cov.iloc[i:tmp, 3] = -9
                    i = tmp
                else: 
                    cov.iloc[i:tmp+1, 3] = -9
                    i = tmp
    res = cov[cov['cov'] >= 0].sort_values(by = ['chr', 'start']).reset_index(drop = True)
    return res

def multi_parse(chromosomes, files, sample, n_processes = 22,
               info_cols = ['EAF', 'INFO_SCORE'], attribute = 'INFO', fmt = 'FORMAT', drop_attribute = True, drop_lst = ['ID', 'QUAL', 'FILTER']):
    manager = multiprocessing.Manager()
    q = manager.Queue()
    processes = []
    for i in range(len(chromosomes)):
        tmp = multiprocessing.Process(target=parse_vcf, args=(files[i], sample, q, info_cols, attribute, fmt, drop_attribute, drop_lst))
        tmp.start()
        processes.append(tmp)
    for process in processes:
        process.join()
    res_lst = []
    while not q.empty():
        res_lst.append(q.get())
    return res_lst

In [3]:
chromosomes = [i for i in range(1,23)]
sample = 'GM8'
files = ['../../GAMCC_oneKG/test/chr' + str(i) + '.vcf.gz' for i in chromosomes]
res = multi_parse(chromosomes, files, sample)
df = concat_vcf(res).sort_values(by = ['CHROM', 'POS'])
df

In [ ]:
cols = ['chr', 'start', 'end', 'cov']
cov = pd.read_csv('IDT0481.bed', sep = '\t', header = None, names = cols)
cov['chr'] = cov['chr'].str.extract(r'(\d+)').astype(int)
cov = cov.sort_values(by = ['chr', 'start'])
region = pd.read_csv('sorted.bed', sep = '\t', header = None, names = cols[:-1])
region = region[region['chr'].isin(['chr' + str(i) for i in range(1,23)])]
region['chr'] = region['chr'].str.extract(r'(\d+)').astype(int)
region = region.sort_values(by = ['chr', 'start'])
subtract_bed_by_chr(cov, region)